# Imports

In [1]:
import pandas as pd

# ...

In [17]:
df = pd.read_csv('../data/felipeneto/comments.csv')
df.head()

,id,video_id,video_title,comment_id,comment_text,comment_author_name,comment_author_channel_id,comment_like_count,comment_publish_date,comment_reply_count,is_reply,parent_comment_id
0,0,1151iRJ9gt4,TEM QUE PROIBIR FESTA DE ANIVERSÁRIO COM ESSE ...,Ugx05FVwoeXg-5FqoiF4AaABAg,Moska: infantil é Voce. EU MORIIIII KKKKKKKKKK...,@jiii6099,UCGyrwQ9TT4ncLw3T7itNpSg,876,2024-09-09T13:03:49Z,13,False,0
1,1,1151iRJ9gt4,TEM QUE PROIBIR FESTA DE ANIVERSÁRIO COM ESSE ...,Ugx05FVwoeXg-5FqoiF4AaABAg.A8A8NA4JDT3A8A9HoMQGAS,@@jiii6099 o Luccas neto 🤔🥲,@Cherry_br啊默认,UCMak6ni2dvZS44uyS-qhVfA,5,2024-09-09T13:11:49Z,0,True,Ugx05FVwoeXg-5FqoiF4AaABAg
2,2,1151iRJ9gt4,TEM QUE PROIBIR FESTA DE ANIVERSÁRIO COM ESSE ...,Ugx05FVwoeXg-5FqoiF4AaABAg.A8A8NA4JDT3A8AWkSOgD6c,A fala de toda criança akakkakakakak <br>Eu amei,@Kit__kathm,UCrqPxKqikKR_LyTaDwtj5DQ,10,2024-09-09T16:36:51Z,0,True,Ugx05FVwoeXg-5FqoiF4AaABAg
3,3,1151iRJ9gt4,TEM QUE PROIBIR FESTA DE ANIVERSÁRIO COM ESSE ...,Ugx05FVwoeXg-5FqoiF4AaABAg.A8A8NA4JDT3A8AdsguL8mF,Kdkskskskskksks eu tbm😂,@VictóriaBeatrisCassianodeMelo,UC_5AhNewMx07HrIZYpq0aRg,5,2024-09-09T17:47:52Z,0,True,Ugx05FVwoeXg-5FqoiF4AaABAg
4,4,1151iRJ9gt4,TEM QUE PROIBIR FESTA DE ANIVERSÁRIO COM ESSE ...,Ugx05FVwoeXg-5FqoiF4AaABAg.A8A8NA4JDT3A8AjRNbhfmH,Kkaqkkaakkskdjrhdjkwkkkkkk eu também😂,@Bruna-us2be,UCqqEio6xA2yHuImh2JV7Lkg,5,2024-09-09T18:36:26Z,0,True,Ugx05FVwoeXg-5FqoiF4AaABAg


## DF by like count

In [16]:
# sorted_df = df.sort_values(by=['comment_like_count', 'comment_reply_count'], ascending=[False, False])
sorted_df = df.sort_values(by=['comment_reply_count','comment_like_count'], ascending=[False, False])

sorted_df.head()

,id,video_id,video_title,comment_id,comment_text,comment_author_name,comment_author_channel_id,comment_like_count,comment_publish_date,comment_reply_count,is_reply,parent_comment_id
7499,7499,kLlhl31GL4M,MAIORES LAPADA SECA DA INTERNET,Ugz3XNhH5nvRkCkpILR4AaABAg,Reuno aq tds as corujas de 2016,@nandaa55,UC4vtP4WcLC3WgvNBVcdDvCg,5942,2023-05-10T13:02:22Z,539,False,0
6836,6836,-iCzLlOOfBk,MITOLOGIAS INCRÍVEIS - EXISTIU?,UgxJON9TmKIANWV_XoJ4AaABAg,Petição para o Felipe falar da Medusa. Levam s...,@rafaelavieira350,UCa72-uq1hd-C2jjRpM2lJKg,3770,2023-05-18T15:57:37Z,492,False,0
3176,3176,SmrtiUP3NDA,MEUS AMIGOS ME COMPRARAM PRESENTES! VAMOS ABRIR…,UgzgBJ3FDgbXLlreIiN4AaABAg,quem não tem amigos que dão presentes levanta ...,@DougNappa,UCazor3Ou7qDnEt9WlYXzHcw,2949,2023-06-29T13:08:22Z,263,False,0
5942,5942,Y10Ygl-0VrI,FIM! MERGULHAMOS NUM LAGO CONGELADO! Los5 - Di...,UgxXSDyV47rJUtaGcFt4AaABAg,Quem concorda que o próximo Los5 tem que ser n...,@marianarocha6121,UC9l4pBufPe0ML3xgViD-MVQ,9210,2023-05-28T18:43:48Z,250,False,0
1232,1232,RZGWBX3U5u0,PICO PARK 2 - O PESADELO VOLTOU #1,UgwoG7M0Dm5IKM5mIv14AaABAg,"&quot;vou te demitir, vou te mandar pra play9&...",@claraaaa564,UChbuE25nqTRpByk_9bZx66g,4087,2024-09-03T23:00:18Z,175,False,0


In [25]:
df[df["parent_comment_id"] == 'UgzgBJ3FDgbXLlreIiN4AaABAg']

,id,video_id,video_title,comment_id,comment_text,comment_author_name,comment_author_channel_id,comment_like_count,comment_publish_date,comment_reply_count,is_reply,parent_comment_id
3177,3177,SmrtiUP3NDA,MEUS AMIGOS ME COMPRARAM PRESENTES! VAMOS ABRIR…,UgzgBJ3FDgbXLlreIiN4AaABAg.9rYKiExJpOh9rYMaVcHynt,Mas a mão tá smp levantada,@hugo.f.12,UC20LIHozUzFenq5-sLlcIzw,27,2023-06-29T13:24:46Z,0,True,UgzgBJ3FDgbXLlreIiN4AaABAg
3178,3178,SmrtiUP3NDA,MEUS AMIGOS ME COMPRARAM PRESENTES! VAMOS ABRIR…,UgzgBJ3FDgbXLlreIiN4AaABAg.9rYKiExJpOh9rYMx2N3VKg,✋🏻,@carolgsp,UC-gDXDZroYHQ765GGzLSWBA,15,2023-06-29T13:27:51Z,0,True,UgzgBJ3FDgbXLlreIiN4AaABAg
3179,3179,SmrtiUP3NDA,MEUS AMIGOS ME COMPRARAM PRESENTES! VAMOS ABRIR…,UgzgBJ3FDgbXLlreIiN4AaABAg.9rYKiExJpOh9rYOW3ddCuu,🤚🏽,@AdrianoLima-ut9ei,UCwN8SMxRNb6CuH8jNTU3nQQ,7,2023-06-29T13:41:30Z,0,True,UgzgBJ3FDgbXLlreIiN4AaABAg
3180,3180,SmrtiUP3NDA,MEUS AMIGOS ME COMPRARAM PRESENTES! VAMOS ABRIR…,UgzgBJ3FDgbXLlreIiN4AaABAg.9rYKiExJpOh9rYO_H9BWpE,Só minha namorada me dá presente 😂,@MELQUICAMPOS,UCVWI0GLD5PdgXpsaV0mYvrQ,13,2023-06-29T13:42:05Z,0,True,UgzgBJ3FDgbXLlreIiN4AaABAg
3181,3181,SmrtiUP3NDA,MEUS AMIGOS ME COMPRARAM PRESENTES! VAMOS ABRIR…,UgzgBJ3FDgbXLlreIiN4AaABAg.9rYKiExJpOh9rYOg6YOGh7,Mas se for com meu dinheiro prefiro continuar ...,@safitrytomaselli1678,UC7wSiYrjxGHOJPpqHnk7CyQ,30,2023-06-29T13:43:01Z,0,True,UgzgBJ3FDgbXLlreIiN4AaABAg
